In [ ]:

#First we import some libraries
#Json for loading and saving the model (optional)
import json
#matplotlib for rendering
import matplotlib.pyplot as plt
#numpy for handeling matrix operations
import numpy as np
#time, to, well... keep track of time
import time
#Python image libarary for rendering
from PIL import Image
#iPython display for making sure we can render the frames
from IPython import display
#seaborn for rendering
import seaborn
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD


#Setup matplotlib so that it runs nicely in iPython
%matplotlib inline
#setting up seaborn
seaborn.set()

class Catch(object):
    """
    Class catch is the actual game.
    In the game, fruits, represented by white tiles, fall from the top.
    The goal is to catch the fruits with a basked (represented by white tiles, this is deep learning, not game design).
    """
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self.state = out

    def _draw_state(self):
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas
        
    def _get_reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1
            else:
                return -1
        else:
            return 0

    def _is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    def observe(self):
        canvas = self._draw_state()
        return canvas.reshape((1, -1))

    def act(self, action):
        self._update_state(action)
        reward = self._get_reward()
        game_over = self._is_over()
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self.state = np.asarray([0, n, m])[np.newaxis]

        
"""
Here we define some variables used for the game and rendering later
"""
#last frame time keeps track of which frame we are at
last_frame_time = 0
#translate the actions to human readable words
translate_action = ["Left","Stay","Right","Create Ball","End Test"]
#size of the game field
grid_size = 10

def display_screen(action,points,input_t):
    #Function used to render the game screen
    #Get the last rendered frame
    global last_frame_time
    print("Action %s, Points: %d" % (translate_action[action],points))
    #Only display the game screen if the game is not over
    if("End" not in translate_action[action]):
        #Render the game with matplotlib
        plt.imshow(input_t.reshape((grid_size,)*2),
               interpolation='none', cmap='gray')
        #Clear whatever we rendered before
        display.clear_output(wait=True)
        #And display the rendering
        display.display(plt.gcf())
    #Update the last frame time
    last_frame_time = set_max_fps(last_frame_time)
    
    
def set_max_fps(last_frame_time,FPS = 16):
    current_milli_time = lambda: int(round(time.time() * 1000))
    sleep_time = 1./FPS - (current_milli_time() - last_frame_time)
    if sleep_time > 0:
        time.sleep(sleep_time)
    return current_milli_time()

In [17]:
import sys
import gym
import pylab
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

EPISODES = 1000000


# This is Policy Gradient agent for the Cartpole
# In this example, we use REINFORCE algorithm which uses monte-carlo update rule
class REINFORCEAgent:
    def __init__(self, state_size, action_size):
        # if you want to see Cartpole learning, then change to True
        self.render = True
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.9
        self.learning_rate = 1e-3
#         self.decay = 0.999
#         self.learning_rate = 0.00035
        self.hidden1, self.hidden2 = 100, 20

        # create model for policy network
        self.model = self.build_model()

        # lists for the states, actions and rewards
        self.states, self.actions, self.rewards = [], [], []

        
    # approximate policy using Neural Network
    # state is input and probability of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(self.hidden1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_uniform'))
        model.add(Dense(self.hidden2, activation='relu', kernel_initializer='glorot_uniform'))
        model.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_uniform'))
        model.summary()
        # Using categorical crossentropy as a loss is a trick to easily
        # implement the policy gradient. Categorical cross entropy is defined
        # H(p, q) = sum(p_i * log(q_i)). For the action taken, a, you set 
        # p_a = advantage. q_a is the output of the policy network, which is
        # the probability of taking the action a, i.e. policy(s, a). 
        # All other p_i are zero, thus we have H(p, q) = A * log(policy(s, a))
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=self.learning_rate))
        return model

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.model.predict(state, batch_size=8).flatten()
#         print(policy)
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # In Policy Gradient, Q function is not available.
    # Instead agent uses sample returns for evaluating policy
    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, len(rewards))):
            running_add = running_add * self.discount_factor + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save <s, a ,r> of each step
    def append_sample(self, state, action, reward):
        self.states.append(state)
        self.rewards.append(reward)
        self.actions.append(action)
    
    def clear_episode(self):
        self.states.clear()
        self.rewards.clear()
        self.actions.clear()

    # update policy network every episode
    def train_model(self):
        episode_length = len(self.states)
        discounted_rewards = self.discount_rewards(self.rewards)
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)
        update_inputs = np.zeros((episode_length, self.state_size))
        advantages = np.zeros((episode_length, self.action_size))

        for i in range(episode_length):
            update_inputs[i] = self.states[i]
#             print(self.actions[i])
            advantages[i][self.actions[i]] = discounted_rewards[i]-1.434

        self.model.fit(update_inputs, advantages, epochs=1, verbose=0)
        self.clear_episode()
#         self.states, self.actions, self.rewards = [], [], []

if __name__ == "__main__":
    state_size = 100
    action_size = 3
    
    #setup environment
    plt.ion()
    # Define environment, game
    env = Catch(grid_size)
    maxs = 0;
    # make REINFORCE agent
    agent = REINFORCEAgent(state_size, action_size)

    scores, episodes, average_reward = [], [], []
    score = 0
    for e in range(EPISODES):
        done = False
        env.reset()
        state = env.observe()
        state = np.reshape(state, [1, state_size])
        while not done:
#             if agent.render:
#                 display_screen(action,score,state)
            
            # get action for the current state and go one step in environment
            action = agent.get_action(state)
#             print(action)   
#             next_state, reward, done, info = env.step(action)
            next_state, reward, done = env.act(action)
            next_state = np.reshape(next_state, [1, state_size])
            reward = float(reward)

            # save the sample <s, a, r> to the memory
            agent.append_sample(state, action, reward)
        
            score += reward
            state = next_state
            if done:
                # every episode, agent learns from sample returns
                agent.train_model()
                average_reward.append(score)
                score = 0
                if len(average_reward)>300: average_reward.pop(0)
                ar = np.mean(average_reward)
                if maxs<ar and e > 300:
                    maxs = ar
                    agent.model.save("./catch_reinforce.h5")
                    print("saved model maxs:", maxs, "caught fruits (300): ",(300*maxs+300)/2)

                print("episode:", e,"reward:",reward,"average reward:",ar)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_32 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_33 (Dense)             (None, 3)                 63        
Total params: 12,183
Trainable params: 12,183
Non-trainable params: 0
_________________________________________________________________
episode: 0 reward: -1.0 average reward: -1.0
episode: 1 reward: 1.0 average reward: 0.0
episode: 2 reward: -1.0 average reward: -0.3333333333333333
episode: 3 reward: 1.0 average reward: 0.0
episode: 4 reward: -1.0 average reward: -0.2
episode: 5 reward: -1.0 average reward: -0.3333333333333333
episode: 6 reward: -1.0 average reward: -0.42857142857142855
episode: 7 reward: 1.0 average reward: -0.25


episode: 129 reward: -1.0 average reward: -0.35384615384615387
episode: 130 reward: -1.0 average reward: -0.35877862595419846
episode: 131 reward: 1.0 average reward: -0.3484848484848485
episode: 132 reward: 1.0 average reward: -0.3383458646616541
episode: 133 reward: 1.0 average reward: -0.3283582089552239
episode: 134 reward: -1.0 average reward: -0.3333333333333333
episode: 135 reward: 1.0 average reward: -0.3235294117647059
episode: 136 reward: -1.0 average reward: -0.3284671532846715
episode: 137 reward: 1.0 average reward: -0.3188405797101449
episode: 138 reward: -1.0 average reward: -0.3237410071942446
episode: 139 reward: 1.0 average reward: -0.3142857142857143
episode: 140 reward: -1.0 average reward: -0.3191489361702128
episode: 141 reward: 1.0 average reward: -0.30985915492957744
episode: 142 reward: -1.0 average reward: -0.3146853146853147
episode: 143 reward: -1.0 average reward: -0.3194444444444444
episode: 144 reward: 1.0 average reward: -0.3103448275862069
episode: 145 

episode: 271 reward: -1.0 average reward: -0.375
episode: 272 reward: 1.0 average reward: -0.36996336996337
episode: 273 reward: -1.0 average reward: -0.3722627737226277
episode: 274 reward: 1.0 average reward: -0.36727272727272725
episode: 275 reward: 1.0 average reward: -0.36231884057971014
episode: 276 reward: 1.0 average reward: -0.3574007220216607
episode: 277 reward: -1.0 average reward: -0.3597122302158273
episode: 278 reward: -1.0 average reward: -0.36200716845878134
episode: 279 reward: -1.0 average reward: -0.36428571428571427
episode: 280 reward: -1.0 average reward: -0.3665480427046263
episode: 281 reward: 1.0 average reward: -0.3617021276595745
episode: 282 reward: -1.0 average reward: -0.36395759717314485
episode: 283 reward: -1.0 average reward: -0.36619718309859156
episode: 284 reward: -1.0 average reward: -0.3684210526315789
episode: 285 reward: 1.0 average reward: -0.36363636363636365
episode: 286 reward: -1.0 average reward: -0.36585365853658536
episode: 287 reward: 

episode: 410 reward: 1.0 average reward: -0.2866666666666667
episode: 411 reward: 1.0 average reward: -0.2866666666666667
episode: 412 reward: 1.0 average reward: -0.28
episode: 413 reward: -1.0 average reward: -0.28
episode: 414 reward: 1.0 average reward: -0.2733333333333333
episode: 415 reward: 1.0 average reward: -0.26666666666666666
episode: 416 reward: 1.0 average reward: -0.26
episode: 417 reward: -1.0 average reward: -0.26
episode: 418 reward: -1.0 average reward: -0.26666666666666666
episode: 419 reward: 1.0 average reward: -0.26666666666666666
episode: 420 reward: 1.0 average reward: -0.26666666666666666
episode: 421 reward: -1.0 average reward: -0.2733333333333333
episode: 422 reward: 1.0 average reward: -0.2733333333333333
episode: 423 reward: 1.0 average reward: -0.26666666666666666
episode: 424 reward: -1.0 average reward: -0.2733333333333333
episode: 425 reward: -1.0 average reward: -0.28
episode: 426 reward: -1.0 average reward: -0.28
episode: 427 reward: -1.0 average r

episode: 554 reward: -1.0 average reward: -0.2733333333333333
episode: 555 reward: -1.0 average reward: -0.2733333333333333
episode: 556 reward: -1.0 average reward: -0.28
episode: 557 reward: 1.0 average reward: -0.2733333333333333
episode: 558 reward: -1.0 average reward: -0.2733333333333333
episode: 559 reward: -1.0 average reward: -0.28
episode: 560 reward: -1.0 average reward: -0.28
episode: 561 reward: -1.0 average reward: -0.28
episode: 562 reward: 1.0 average reward: -0.2733333333333333
episode: 563 reward: 1.0 average reward: -0.26666666666666666
episode: 564 reward: -1.0 average reward: -0.26666666666666666
episode: 565 reward: -1.0 average reward: -0.26666666666666666
episode: 566 reward: 1.0 average reward: -0.26
episode: 567 reward: 1.0 average reward: -0.26
episode: 568 reward: 1.0 average reward: -0.25333333333333335
episode: 569 reward: -1.0 average reward: -0.25333333333333335
episode: 570 reward: 1.0 average reward: -0.24666666666666667
episode: 571 reward: -1.0 avera

...

episode: 14521 reward: 1.0 average reward: 0.3933333333333333
episode: 14522 reward: -1.0 average reward: 0.3933333333333333
episode: 14523 reward: 1.0 average reward: 0.3933333333333333
episode: 14524 reward: -1.0 average reward: 0.38666666666666666
episode: 14525 reward: 1.0 average reward: 0.38666666666666666
episode: 14526 reward: 1.0 average reward: 0.3933333333333333
episode: 14527 reward: 1.0 average reward: 0.3933333333333333
episode: 14528 reward: 1.0 average reward: 0.4
episode: 14529 reward: -1.0 average reward: 0.3933333333333333
episode: 14530 reward: 1.0 average reward: 0.3933333333333333
episode: 14531 reward: 1.0 average reward: 0.4
episode: 14532 reward: 1.0 average reward: 0.4066666666666667
episode: 14533 reward: 1.0 average reward: 0.4066666666666667
episode: 14534 reward: 1.0 average reward: 0.4066666666666667
episode: 14535 reward: -1.0 average reward: 0.4066666666666667
episode: 14536 reward: 1.0 average reward: 0.4066666666666667
episode: 14537 reward: 1.0 avera

episode: 14672 reward: -1.0 average reward: 0.43333333333333335
episode: 14673 reward: 1.0 average reward: 0.43333333333333335
episode: 14674 reward: -1.0 average reward: 0.4266666666666667
episode: 14675 reward: -1.0 average reward: 0.42
episode: 14676 reward: 1.0 average reward: 0.4266666666666667
episode: 14677 reward: -1.0 average reward: 0.4266666666666667
episode: 14678 reward: 1.0 average reward: 0.4266666666666667
episode: 14679 reward: -1.0 average reward: 0.4266666666666667
episode: 14680 reward: -1.0 average reward: 0.4266666666666667
episode: 14681 reward: 1.0 average reward: 0.4266666666666667
episode: 14682 reward: -1.0 average reward: 0.42
episode: 14683 reward: -1.0 average reward: 0.41333333333333333
episode: 14684 reward: 1.0 average reward: 0.41333333333333333
episode: 14685 reward: -1.0 average reward: 0.4066666666666667
episode: 14686 reward: 1.0 average reward: 0.41333333333333333
episode: 14687 reward: -1.0 average reward: 0.4066666666666667
episode: 14688 reward

episode: 14821 reward: 1.0 average reward: 0.32
episode: 14822 reward: -1.0 average reward: 0.32
episode: 14823 reward: 1.0 average reward: 0.32
episode: 14824 reward: 1.0 average reward: 0.32666666666666666
episode: 14825 reward: 1.0 average reward: 0.32666666666666666
episode: 14826 reward: 1.0 average reward: 0.32666666666666666
episode: 14827 reward: -1.0 average reward: 0.32
episode: 14828 reward: -1.0 average reward: 0.31333333333333335
episode: 14829 reward: -1.0 average reward: 0.31333333333333335
episode: 14830 reward: 1.0 average reward: 0.31333333333333335
episode: 14831 reward: 1.0 average reward: 0.31333333333333335
episode: 14832 reward: 1.0 average reward: 0.31333333333333335
episode: 14833 reward: 1.0 average reward: 0.31333333333333335
episode: 14834 reward: 1.0 average reward: 0.31333333333333335
episode: 14835 reward: -1.0 average reward: 0.31333333333333335
episode: 14836 reward: 1.0 average reward: 0.31333333333333335
episode: 14837 reward: 1.0 average reward: 0.31

episode: 14969 reward: 1.0 average reward: 0.24
episode: 14970 reward: 1.0 average reward: 0.24
episode: 14971 reward: -1.0 average reward: 0.24
episode: 14972 reward: 1.0 average reward: 0.24666666666666667
episode: 14973 reward: 1.0 average reward: 0.24666666666666667
episode: 14974 reward: -1.0 average reward: 0.24666666666666667
episode: 14975 reward: -1.0 average reward: 0.24666666666666667
episode: 14976 reward: -1.0 average reward: 0.24
episode: 14977 reward: 1.0 average reward: 0.24666666666666667
episode: 14978 reward: 1.0 average reward: 0.24666666666666667
episode: 14979 reward: 1.0 average reward: 0.25333333333333335
episode: 14980 reward: -1.0 average reward: 0.25333333333333335
episode: 14981 reward: 1.0 average reward: 0.25333333333333335
episode: 14982 reward: 1.0 average reward: 0.26
episode: 14983 reward: 1.0 average reward: 0.26666666666666666
episode: 14984 reward: 1.0 average reward: 0.26666666666666666
episode: 14985 reward: 1.0 average reward: 0.2733333333333333


KeyboardInterrupt: 

In [18]:
print(maxs)


0.6666666666666666


In [22]:
from keras.models import load_model
state_size = 100
action_size = 3
env = Catch(grid_size)
# make REINFORCE agent
agent = load_model('catch_reinforce.h5')
score = 0
for e in range(500):
    done = False
    env.reset()
    state = env.observe()
    state = np.reshape(state, [1, state_size])
    print(str(e)+" "+str(score))
    while not done:
        policy = agent.predict(state)
        action = np.argmax(policy)
        next_state, reward, done = env.act(action)
        next_state = np.reshape(next_state, [1, state_size])
        score = score + reward
#         display_screen(action,score,next_state)
        state = next_state
        

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 7
10 8
11 9
12 10
13 11
14 12
15 13
16 14
17 15
18 16
19 17
20 16
21 15
22 16
23 17
24 18
25 19
26 20
27 21
28 22
29 23
30 22
31 23
32 24
33 25
34 26
35 27
36 28
37 29
38 30
39 31
40 32
41 33
42 34
43 33
44 34
45 35
46 36
47 37
48 38
49 39
50 40
51 41
52 42
53 43
54 44
55 45
56 46
57 47
58 48
59 49
60 50
61 51
62 52
63 51
64 52
65 53
66 52
67 53
68 52
69 53
70 54
71 55
72 56
73 57
74 58
75 57
76 58
77 57
78 58
79 59
80 60
81 61
82 62
83 63
84 64
85 65
86 66
87 67
88 68
89 69
90 70
91 71
92 72
93 71
94 72
95 73
96 74
97 75
98 74
99 73
100 74
101 75
102 76
103 77
104 78
105 79
106 78
107 79
108 80
109 79
110 78
111 79
112 80
113 81
114 82
115 83
116 84
117 85
118 84
119 85
120 86
121 87
122 88
123 89
124 90
125 91
126 92
127 93
128 94
129 95
130 96
131 97
132 98
133 99
134 100
135 101
136 102
137 101
138 102
139 101
140 102
141 101
142 102
143 103
144 104
145 105
146 106
147 107
148 108
149 109
150 108
151 107
152 108
153 109
154 110
155 111
156 112
